# 运行YOLO分割模型，给数据标记掩码



## 查看根目录下需要运行的文件夹

In [ ]:
from pathlib import Path

if __name__ == "__main__":
    # === 1) 给定一个根目录 ===
    root_dir = Path("/workspace/models/SAHI/run_v7")
    # end_with = "_sliced"
    end_with = "_data"

    # === 2) 遍历所有子目录 ===
    sub_dirs = list(root_dir.glob("**/*" + end_with))


    if not sub_dirs:
        print(f"没有找到 *{end_with} 目录")
        exit(0)

    print(f"找到以下 {end_with} 数据集：")
    for i, d in enumerate(sub_dirs):
        print(f"[{i}] {d}")

    # === 3) 让你选择要跑的目录 ===
    idx_str = input("请输入要处理的编号 (多个用逗号分隔, 回车默认全选): ").strip()
    if idx_str:
        indices = [int(x) for x in idx_str.split(",")]
        chosen_dirs = [sub_dirs[i] for i in indices]
    else:
        chosen_dirs = sub_dirs
    
    print(f"将处理以下 {end_with} 目录：")
    for i, d in enumerate(chosen_dirs):
        print(f"- {i+1}. {d}")
        
    # 如果有的文件夹的raw_data里面没有图片，就移除
    chosen_dirs = [d for d in chosen_dirs if (d.parent / "raw_data").exists() and any((d.parent / "raw_data").glob("*.jpg"))]
    if not chosen_dirs:
        print(f"没有找到包含图片的 *{end_with} 目录")
        exit(0)

找到以下 _data 数据集：
[0] /workspace/models/SAHI/run_v7/ms1_0726-0809_11/raw_data
[1] /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data
[2] /workspace/models/SAHI/run_v7/air1_0826-0909_21_no/raw_data
[3] /workspace/models/SAHI/run_v7/air1_0909-0923_02_wait/raw_data
[4] /workspace/models/SAHI/run_v7/air1_0923-1007_01_wait/raw_data
[5] /workspace/models/SAHI/run_v7/air2_0729-0813_04/raw_data
[6] /workspace/models/SAHI/run_v7/air2_0826-0909_25_no/raw_data
[7] /workspace/models/SAHI/run_v7/air2_0923-1007_02_wait/raw_data
[8] /workspace/models/SAHI/run_v7/jeff_0613-0624_04_ok/raw_data
[9] /workspace/models/SAHI/run_v7/jeff_0624-0702_01_ok/raw_data
[10] /workspace/models/SAHI/run_v7/jeff_0730-0813_01/raw_data
[11] /workspace/models/SAHI/run_v7/lloyd_0603-0618_31/raw_data
[12] /workspace/models/SAHI/run_v7/lloyd_0715-0729_04/raw_data
[13] /workspace/models/SAHI/run_v7/lloyd_0826-0909_04/raw_data
[14] /workspace/models/SAHI/run_v7/ms1_0605-0621_40/raw_data
[15] /workspace/models/SAHI/run_v7/ms

## 分割，导出结果

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import os
import orjson as jsonlib

# Load a pretrained YOLO11n model
model = YOLO("/workspace/models/best_model/yolo11n-seg-best.pt")

for d in chosen_dirs:
    print(f"\n=== 处理目录: {d} ===")   

    source = str(d) + "_sliced"

    if not any(Path(source).iterdir()):
        print(f"跳过空目录: {source}")
        continue

    # Run inference on 'bus.jpg' with argument
    results = model.predict(source, imgsz=640, conf=0.25, iou=0.45, device='cpu')

    _dumps = lambda obj: jsonlib.dumps(obj, option=jsonlib.OPT_INDENT_2 | jsonlib.OPT_NON_STR_KEYS)
    _loads = jsonlib.loads


    # 输出目录
    for result in results:
        # 1) 解析 YOLO 的 JSON（string -> list[dict]）
        det_list = _loads(result.to_json())

        # 2) 准备 LabelMe 顶层字段
        h, w = map(int, result.orig_shape[:2])
        img_name = os.path.basename(getattr(result, "path", "")) or "image.png"

        shapes = []
        for det in det_list:
            seg = det.get("segments", {})
            xs, ys = seg.get("x", []), seg.get("y", [])
            if not xs or not ys:
                continue
            # 组装 [ [x,y], ... ]；转 float 以得到 146.0 这种形式
            points = [[float(x), float(y)] for x, y in zip(xs, ys)]

            shapes.append({
                "label": det.get("name", ""),
                "score": float(det.get("confidence", 0.0)),
                "points": points,
                "shape_type": "polygon",
            })

        labelme_obj = {
            "shapes": shapes,
            "imagePath": img_name,
            "imageHeight": h,
            "imageWidth": w,
        }

        # 3) 写文件：与图片同名 .json
        out_path = source + f"/{Path(img_name).stem}.json"
        Path(out_path).write_bytes(_dumps(labelme_obj))


    print(f"✅ Done. Saved to: {source}")



=== 处理目录: /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data ===

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/9387 /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data_sliced/0805_1254_580_0_0_640_640.jpg: 640x640 (no detections), 40.9ms
image 2/9387 /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data_sliced/0805_1254_580_0_1024_640_1664.jpg: 640x640 (no detections), 31.0ms
image 3/9387 /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data_sliced/0805_1254_580_0_1536_640_21

: 